### LIBRARIES

In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from lazypredict.Supervised import LazyClassifier

In [11]:
data = pd.read_csv("D:/University/2022-2023\HK2/thuThapVaTienXuLyDuLieu/fat_preprocessing/comment_final.csv", encoding = 'utf-8')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85999 entries, 0 to 85998
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      85999 non-null  int64 
 1   processed_text  81150 non-null  object
 2   class           85999 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.0+ MB


In [12]:
data['class'].value_counts()

positive    76005
negative     5666
neutral      4328
Name: class, dtype: int64

In [13]:
df_sub= data.sample(frac=0.02)

In [14]:
source = df_sub['processed_text']
type(source)

pandas.core.series.Series

In [15]:
target = df_sub['class']
type(target)

pandas.core.series.Series

### Lazy Predict with 3 class

In [16]:
target = target.replace("negative", 0)
target = target.replace("neutral", 1)
target = target.replace("positive", 2)

### TF_IDF

In [17]:
# Create a TfidfVectorizer object
tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=0.02)

In [18]:
# Fit the TfidfVectorizer object to the data
x = tfidf_vectorizer.fit(df_sub['processed_text'].values.astype('U'))

In [19]:
# Transform the data
X = tfidf_vectorizer.transform(df_sub['processed_text'].values.astype('U'))

In [20]:
### BUG: 
# This is probably because you are using an older scikit-learn version than the one this code was written for.
# get_feature_names_out is a method of the class sklearn.feature_extraction.text.TfidfVectorizer since scikit-learn 1.0. Previously, there was a similar method called get_feature_names.
# So you should update your scikit-learn package, or use the old method (not recommended).
# Create a new dataframe with the transformed data
df_new = pd.DataFrame(X.toarray(), columns=tfidf_vectorizer.get_feature_names_out())

In [21]:
X = X.toarray()

In [22]:
y = np.array(target)

TEST = 20%, TRAIN = 80%

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20) 

In [31]:
clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
models, predictions = clf.fit(X_train, X_test, y_train, y_test)

100%|██████████| 29/29 [00:04<00:00,  6.91it/s]


In [34]:
models

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
NearestCentroid,0.60,0.46,None,0.68,0.01
PassiveAggressiveClassifier,0.76,0.44,None,0.78,0.02
LabelSpreading,0.73,0.43,None,0.75,0.19
LabelPropagation,0.72,0.42,None,0.75,0.17
DecisionTreeClassifier,0.79,0.41,None,0.78,0.03
GaussianNB,0.19,0.40,None,0.22,0.01
LinearDiscriminantAnalysis,0.85,0.39,None,0.82,0.02
BernoulliNB,0.86,0.38,None,0.82,0.01
KNeighborsClassifier,0.86,0.37,None,0.81,0.03


Chọn được những model có hiệu suất tốt sau:

ExtraTreesClassifier

RidgeClassifier

KNeighborsClassifier

RandomForestClassifier

LogisticRegression

### Resampling

In [19]:
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE

In [20]:
smt = SMOTE()
X_smt, y_smt = smt.fit_resample(X,y)

In [21]:
X_train_smt, X_test_smt, y_train_smt, y_test_smt = train_test_split(X_smt, y_smt, test_size=0.20) 

In [28]:
clf2 = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=None)
modelsresampling, predictions = clf2.fit(X_train_smt, X_test_smt, y_train_smt, y_test_smt)

100%|██████████| 29/29 [00:14<00:00,  2.02it/s]


In [29]:
modelsresampling

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.88,0.88,None,0.88,0.58
RandomForestClassifier,0.87,0.87,None,0.87,0.50
LGBMClassifier,0.87,0.87,None,0.87,0.34
XGBClassifier,0.85,0.85,None,0.85,0.50
BaggingClassifier,0.83,0.83,None,0.83,0.24
SVC,0.82,0.82,None,0.82,0.76
DecisionTreeClassifier,0.81,0.81,None,0.81,0.05
ExtraTreeClassifier,0.80,0.81,None,0.80,0.02
LabelSpreading,0.79,0.80,None,0.79,1.36


### SMOTE for 3 classes

In [22]:
models_smt, predictions_smt = clf.fit(X_train_smt, X_test_smt, y_train_smt, y_test_smt)

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


100%|██████████| 29/29 [00:14<00:00,  1.98it/s]


In [23]:
models_smt

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.88,0.88,None,0.88,0.58
RandomForestClassifier,0.87,0.87,None,0.87,0.52
LGBMClassifier,0.87,0.87,None,0.87,0.35
XGBClassifier,0.85,0.85,None,0.85,0.50
BaggingClassifier,0.83,0.83,None,0.83,0.24
SVC,0.82,0.82,None,0.82,0.77
DecisionTreeClassifier,0.81,0.81,None,0.81,0.05
ExtraTreeClassifier,0.80,0.81,None,0.80,0.02
LabelSpreading,0.79,0.80,None,0.79,1.30


In [45]:
models_smt2, predictions_smt2 = clf.fit(X_train_smt2, X_test_smt2, y_train_smt2, y_test_smt2)

'tuple' object has no attribute '__name__'
Invalid Classifier(s)


100%|██████████| 29/29 [00:05<00:00,  5.64it/s]


### SMOTE for 2 CLasses

In [46]:
models_smt2

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
ExtraTreesClassifier,0.83,0.83,0.83,0.83,0.39
XGBClassifier,0.82,0.82,0.82,0.82,0.14
SVC,0.82,0.81,0.81,0.82,0.32
NuSVC,0.81,0.81,0.81,0.81,0.36
RandomForestClassifier,0.81,0.81,0.81,0.81,0.38
LGBMClassifier,0.81,0.81,0.81,0.81,0.13
BaggingClassifier,0.78,0.78,0.78,0.78,0.16
ExtraTreeClassifier,0.78,0.77,0.77,0.77,0.02
DecisionTreeClassifier,0.77,0.77,0.77,0.77,0.04


## Kết luận

- đối với mô hình sử dụng 3 biến, ta sẽ không sử dụng resampling do kết quả không đuọc cải thiện
- đối với mô hình 2 biến thì chúng ta sẽ resampling do kết quae được cải thiện rất tốt

#### Model preslection
- dựa trên tiêu chi theo thứ tự Accuracy và time, sau đó là ROC và F1

##### đối với 3 class
- RandomForestClassifier
- KNeighborsClassifier
- LogisticRegression